**Instead of using statistical models like bigrams or trigrams, we will use a Multi-Layer Perceptron (MLP). This is because the array matrix required for storing letter combinations grows exponentially as 26^x, where x is the number of letter combinations. Since 26 represents the number of letters in the alphabet, this approach would require excessive computation and memory. Using a neural network like an MLP is a more efficient alternative.**

<div align = 'center'>
    <img src="./MLP_Arch.png" width="500">
<div>


---

# Step 1 Batch/Input Layer

In [121]:
import torch 
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [122]:
#Read in txt file
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [123]:
len(words)

32033

In [124]:
# Convert the alphabet to a index integer using mapping 
chars = sorted(list(set(''.join(words))))
# print(chars)
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0 
itos = {i:s for s,i in stoi.items()}
print(stoi)
print(itos)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


For this following FUnction we are creating a dataset based off of indexs based on the previous mapping and storing them as tensors for each word. For example there is 5 words, so we will end up with a dataset of 5 empty arrays [. . .] and then a variation of all the words as shown in the print out of the funciton: 

In [125]:
# This is the block size for how big the input is going to be for the MLP
block_size = 3

X,y = [],[] # X is the input, y is the Label
count = 5
for w in words[:5]: 
    print(w)
    context = [0] * block_size
    count += len(w)
    for ch in w + '.': 
        ix = stoi[ch]
        # print('ch:', ch)
        # print('ix', ix)
        X.append(context)
        # print('context', context)
        y.append(ix)
        print(''.join([itos[i] for i in context]),'----->', itos[ix])
        context = context[1:] + [ix] 
        
print(count)
X = torch.tensor(X)
Y = torch.tensor(y)

emma
... -----> e
..e -----> m
.em -----> m
emm -----> a
mma -----> .
olivia
... -----> o
..o -----> l
.ol -----> i
oli -----> v
liv -----> i
ivi -----> a
via -----> .
ava
... -----> a
..a -----> v
.av -----> a
ava -----> .
isabella
... -----> i
..i -----> s
.is -----> a
isa -----> b
sab -----> e
abe -----> l
bel -----> l
ell -----> a
lla -----> .
sophia
... -----> s
..s -----> o
.so -----> p
sop -----> h
oph -----> i
phi -----> a
hia -----> .
32


### Function Summary:

This function converts words into training data by mapping characters to indices and storing them as tensors.  

- **Block Size (`3`)**: Defines how many previous characters form the input.  
- **`X` (Input) & `y` (Labels)**:  
  - `X` stores context windows of size 3.  
  - `y` stores the next character for each window.  
- **Process**:  
  1. Start with an empty context (`[0, 0, 0]`).  
  2. Slide through each word (plus `.` at the end).  
  3. Store the context in `X` and the next character in `y`.  
  4. Shift the context and repeat.  
- **Example (`"emma"`)**:  
  ```
  ...  → e  
  ..e  → m  
  .em  → m  
  emm  → a  
  mma  → .  
  ```  
- Finally, `X` and `y` are converted to PyTorch tensors.

In [126]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [127]:
X[:5], Y[:5] # Printing the first 5 rows of the (X) (Y) tensor

# X defines the sequence of characters, and Y defines the next character prediction in the sequence

(tensor([[ 0,  0,  0],
         [ 0,  0,  5],
         [ 0,  5, 13],
         [ 5, 13, 13],
         [13, 13,  1]]),
 tensor([ 5, 13, 13,  1,  0]))

As discussed before the dataset (X) resulted in 32 total rows, 5 empty intialization rows, and 27 variation rows that determine the next pattern. The block size determines how many characters will be remebered in the sequence. 

In [128]:
# C is the randome embedding matrix, with 27 rows and 2 columns
C = torch.randn((27,2))

# Embedding the input tensor X using the embedding matrix C
embedding = C[X]
embedding.shape

torch.Size([32, 3, 2])

**Embedding Summary**  

- **Embeddings map characters to vectors** in a continuous space instead of treating them as separate symbols.  
- **Initially random**, embeddings get optimized during training to capture character relationships.  
- **Example:** A sequence `[0,0,5]` (where `5 = e`) retrieves vectors from an embedding matrix.  
- **Training adjusts these vectors** so similar characters end up closer in space.  
- **Benefit:** Helps the model understand patterns and improves predictions without manual rules.

<div align = 'center'>
    <img src="./2d_Vector_Embedding.png" width="500">
<div>


---

# Step 2: Hidden Layers

In [129]:
W1 = torch.randn((6,100)) # This is the weights that will be input to the first layer
b1 = torch.randn((100)) # This is the bias that will be input to the first layer

We flatten the embeddings from `torch.Size([32, 3, 2]) → torch.Size([32, 6])` to make them compatible for weight multiplication. This is done using `torch.cat`, which concatenates the matrix into a single vector per sample.

Method 1

In [130]:
torch.cat([embedding[:, 0, :], embedding[:, 1, :], embedding[:, 2, :]], 1).shape # Inefficent way of doing it due to new memory being created

torch.Size([32, 6])

If the block size changes (e.g., from 3 to 5), the current approach would need manual updates. To avoid hardcoding, we use `torch.unbind`, which dynamically unravels the tensor, making the code more flexible.

In [131]:
# Use of torch.unbind to 
# torch.unbind(embedding, 1)
# This is equivalent to [embedding[:, 0, :], embedding[:, 1, :], embedding[:, 2, :]

torch.cat(torch.unbind(embedding, 1), 1).shape
# We get the same result as above and it is dynamic for block size

torch.Size([32, 6])

Mehtod 2 More efficent

We use `torch.view()` to reshape the tensor into any desired dimensions. For example, it allows us to convert a tensor from `torch.Size([32, 3, 2])` to `torch.Size([32, 6])` dynamically, making it adaptable to different input shapes.

In [132]:
embedding.view(32,6) # This allows us to make the embedding matrix in a different shape
embedding.view(32,6).shape
# Testing similarity 
# embedding.view(32,6) == torch.cat(torch.unbind(embedding, 1), 1)

torch.Size([32, 6])

In [133]:
# So to multiply the weights with inputs 
h = embedding.view(32,6) @ W1 + b1
h

tensor([[ 0.4617,  1.8559,  0.6170,  ...,  0.7137, -1.3758, -0.3990],
        [-0.1530,  0.7819,  1.3047,  ...,  1.5190,  1.4813, -0.0909],
        [-0.8460,  3.6035,  0.5447,  ...,  0.0594, -0.5832,  1.6827],
        ...,
        [-3.0297,  3.1039,  0.9400,  ..., -0.0989,  3.7681,  2.0138],
        [-4.7763, -1.1197, -3.0604,  ...,  1.1071,  2.9060,  0.9937],
        [-1.6010,  3.0267, -1.0810,  ..., -0.2436, -2.4014,  3.0303]])

- **Multiplies** the **32 input vectors (each of size 6)** with **100 random intialized weights** from `W1 (6, 100)`.  
- **Adds bias `b1`** for each of the 32 inputs, resulting in a final shape of `torch.Size([32, 100])`.  

This transforms the input into a **100-dimensional hidden representation** for each sample.

Instead of hardcoding 32 as the batch size in view(32,6), we can use -1 to make the code adaptable to any batch size. (-1) tells PyTorch to automatically infer the correct batch size based on the input. Input Size = The number of features per sample, Batch Size = The number of samples processed at once in a single forward pass.

In [134]:
# Using -1 for dynamic block size
h = embedding.view(-1,6) @ W1 + b1
h

tensor([[ 0.4617,  1.8559,  0.6170,  ...,  0.7137, -1.3758, -0.3990],
        [-0.1530,  0.7819,  1.3047,  ...,  1.5190,  1.4813, -0.0909],
        [-0.8460,  3.6035,  0.5447,  ...,  0.0594, -0.5832,  1.6827],
        ...,
        [-3.0297,  3.1039,  0.9400,  ..., -0.0989,  3.7681,  2.0138],
        [-4.7763, -1.1197, -3.0604,  ...,  1.1071,  2.9060,  0.9937],
        [-1.6010,  3.0267, -1.0810,  ..., -0.2436, -2.4014,  3.0303]])

---

# Step 3: Softmax Layer

In [135]:
W2 = torch.randn((100,27)) 
b2 = torch.randn((27))

In [136]:
logits = h @ W2 + b2 # Hidden layers are multiplied by weights of softmax layer and bias are added 
logits.shape

torch.Size([32, 27])

In [137]:
counts = logits.exp() # Taking the exponential of the logits 

In [138]:
# Normalizing the counts to get the probability of each character
prob = counts / counts.sum(1, keepdim=True)
print(prob.shape) # should return 32, 27, for 32 sameples and 27 characters

print(prob[0].sum()) # The sume of the probabilities should be 1

torch.Size([32, 27])
tensor(1.0000)


In [139]:
prob[torch.arange(32), Y] # This is the probability of the correct character

tensor([2.1675e-14, 3.7844e-18, 1.5153e-33, 1.4448e-25, 4.0409e-21, 4.2833e-13,
        1.6227e-26, 9.7612e-29, 1.1825e-22, 1.4827e-25, 5.3273e-23, 7.4934e-31,
        7.6476e-24, 1.1907e-29, 7.7872e-27, 1.2672e-25, 2.6641e-30, 6.3930e-16,
        4.2929e-13, 3.0654e-22, 6.8697e-22, 2.0577e-29, 2.6431e-10, 1.1858e-14,
        1.3436e-20, 1.5203e-26, 4.7458e-16, 3.3946e-16, 1.8598e-06, 1.0189e-27,
        6.2563e-17, 1.1086e-22])

In [140]:
loss = - prob[torch.arange(32), Y].log().mean() # This is the loss function
loss

tensor(48.7052)

---

# Cleaned Up Full MLP

In [142]:
X.shape, Y.shape # data set 

(torch.Size([32, 3]), torch.Size([32]))

In [ ]:
# Step 1
g = torch.Generator().manual_seed(494949) # Seed for reproducibility
C = torch.rand((27,2), generator=g) # Random embedding matrix

#Step 2
W1 = torch.rand((6,100), generator=g) # Random weights for the first layer
b1 = torch.rand((100), generator=g) # Random bias for the first layer

#Step 3
W2 = torch.rand((100,27), generator=g) # Random weights for the Softmax layer
b2 = torch.rand((27), generator=g) # Random bias for the Softmax layer
params = [C, W1, b1, W2, b2] # Parameters for the model

In [146]:
print('Total Number of Parameters: ', sum(p.nelement() for p in params)) # Total number of parameters in the model

Total Number of Parameters:  3481


In [147]:
embedding = C[X] # Embedding the input tensor X using the embedding matrix C
h = torch.tanh(embedding.view(-1,6) @ W1 + b1) # Hidden layer
logits = h @ W2 + b2 # Logits
counts = logits.exp() # Exponential of the logits
prob = counts / counts.sum(1, keepdim=True) # Normalizing the counts
loss = - prob[torch.arange(32), Y].log().mean() # Loss function
loss 

tensor(6.6207)

Instead of manually implementing the **cross-entropy loss function**, we use PyTorch’s built-in `torch.nn.functional.cross_entropy()`, which is more **optimized and efficient**.  

Unlike a manual implementation, this **does not create extra tensors** for computation. Instead, it clusters operations together, making it **faster** and more **memory-efficient**.  

In [150]:
for p in params: 
    p.requires_grad_() # Setting the requires_grad to True for all the parameters

In [ ]:

# Training the model for 100 epochs
for _ in range(1000): 
    # Forward pass
    embedding = C[X] # Embedding the input tensor X using the embedding matrix C
    h = torch.tanh(embedding.view(-1,6) @ W1 + b1) # Hidden layer
    logits = h @ W2 + b2 # Logits
    loss = F.cross_entropy(logits, Y) # Cross entropy loss function
    
    # Backward pass (Gradient descent)
    for p in params: 
        p.grad = None
    loss.backward()

    #update the parameters
    for p in params: 
        p.data += -0.1 * p.grad
    
print(loss.item())

0.26137614250183105


Even with just **5 words**, the loss dropped from **6.6207 → 0.2613**, but **not to 0**.  

This happens because the **first character of each word is unpredictable**—the model starts with an **empty context**, meaning **any letter has an equal probability of being the first prediction**.  

# Now Training on the full dataset

In [174]:
# This is the block size for how big the input is going to be for the MLP
block_size = 3

X,y = [],[] # X is the input, y is the Label

for w in words: 
    context = [0] * block_size
    for ch in w + '.': 
        ix = stoi[ch]
        X.append(context)
        y.append(ix)
        context = context[1:] + [ix] 
        
X = torch.tensor(X)
Y = torch.tensor(y)

In [175]:
X.shape, X.dtype, Y.shape, Y.dtype
# Now we have 228146 samples and 3 characters in each sample

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [176]:
# Step 1
g = torch.Generator().manual_seed(242424) # Seed for reproducibility
C = torch.rand((27,2), generator=g) # Random embedding matrix

#Step 2
W1 = torch.rand((6,100), generator=g) # Random weights for the first layer
b1 = torch.rand((100), generator=g) # Random bias for the first layer

#Step 3
W2 = torch.rand((100,27), generator=g) # Random weights for the Softmax layer
b2 = torch.rand((27), generator=g) # Random bias for the Softmax layer
params = [C, W1, b1, W2, b2] # Parameters for the model

In [177]:
for p in params: 
    p.requires_grad_() # Setting the requires_grad to True for all the parameters

In [179]:

# Training the model for 100 epochs
for _ in range(10): 
    # Forward pass
    embedding = C[X] # Embedding the input tensor X using the embedding matrix C
    h = torch.tanh(embedding.view(-1,6) @ W1 + b1) # Hidden layer
    logits = h @ W2 + b2 # Logits
    loss = F.cross_entropy(logits, Y) # Cross entropy loss function
    print(loss.item())
    # Backward pass (Gradient descent)
    for p in params: 
        p.grad = None
    loss.backward()

    #update the parameters
    for p in params: 
        p.data += -0.1 * p.grad
    


2.7892494201660156
2.788184881210327
2.787095546722412
2.7859785556793213
2.784837245941162
2.7836713790893555
2.782482862472534
2.7812705039978027
2.780039072036743
2.77878737449646


Notice that this is taking too long to compute so it is best practice to just do backward pass on mini batches instead of every epoch